In [20]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [22]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [23]:
import transformers

print(transformers.__version__)

4.27.4


In [24]:
from datasets import load_dataset
datasets = load_dataset('reddit')

  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
datasets['train'][0]

{'author': 'raysofdarkmatter',
 'body': "I think it should be fixed on either UTC standard or UTC+1 year around, with the current zone offsets.\n\nMoving timescales add a lot of complexity to the implementation of timekeeping systems and have [dubious value]( \n\nI think seasonal shifting time made sense in the pre-electric past, when timekeeping was more flexible and artificial light was inefficient and often dangerous. \n\nNow we have machines that work easily with simple timekeeping rules, and it's more beneficial to spend a small amount on energy for lighting, and save the larger cost of engineering things to work with the complex timekeeping rules, as well as saving the irritation to humans.\n\nLighting has gotten much more efficient over time; we can squeeze out a lot more photons per unit of energy from a 2012 CFL or LED than a candle could in 1780, or a lightbulb could in 1950. \n\nThere's a lot of room for improvement in how we use lights as well; as lighting control gets more

In [26]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=2):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [27]:
show_random_elements(datasets["train"])

,author,body,normalizedBody,subreddit,subreddit_id,id,content,summary
0,kittkatt0,"No, this is not quite correct. Depending on the type of fish, the mercury problem will scale differently; the larger size + longer lifespan the fish is, the more mercury the fish will ingest. W/regards to canned tuna, this primarily refers to albacore tuna (white tuna) which has 35ppm of mercury. They recommend only 2 cans a week for pregnant moms and babies. Light tuna (aka flaked or chunked tuna) only has 1ppm of mercury since the species of fish is much smaller and has a shorter lifespan.\n\ntl;dr - mercury issues don't apply to flaked/chunked tuna, only solid/white tuna.","No, this is not quite correct. Depending on the type of fish, the mercury problem will scale differently; the larger size + longer lifespan the fish is, the more mercury the fish will ingest. W/regards to canned tuna, this primarily refers to albacore tuna (white tuna) which has 35ppm of mercury. They recommend only 2 cans a week for pregnant moms and babies. Light tuna (aka flaked or chunked tuna) only has 1ppm of mercury since the species of fish is much smaller and has a shorter lifespan. \n tl;dr - mercury issues don't apply to flaked/chunked tuna, only solid/white tuna. \n",Fitness,t5_2qhx4,c4a2pqb,"No, this is not quite correct. Depending on the type of fish, the mercury problem will scale differently; the larger size + longer lifespan the fish is, the more mercury the fish will ingest. W/regards to canned tuna, this primarily refers to albacore tuna (white tuna) which has 35ppm of mercury. They recommend only 2 cans a week for pregnant moms and babies. Light tuna (aka flaked or chunked tuna) only has 1ppm of mercury since the species of fish is much smaller and has a shorter lifespan.","mercury issues don't apply to flaked/chunked tuna, only solid/white tuna."
1,SXEatPSU,"And I need some help. At the moment, I'm worried some of the strategies I've enabled in it aren't as viable as I may think they are or as well supported as I think. In addition, I feel like I may be missing out on some amazing cards that could make for really interesting play. Because I want it to be playable with any of the newer players that might show up, I've limited it to relatively recent cards, but I'm afraid that has had some sort of a negative effect on the quality of the cards I've included if I overlooked anything good.\n\n[Here]( a link to what I've got so far. Please let me know if you have any cards that have performed well in your cubes that I may want to include, though keep in mind this is commons only. I want to create a drafting experience where there are several legitimate decks that could win, though pauper does limit how relevant some of the more 'usual' cube archetypes fare.\n\nAt the moment one of the biggest problems I've noticed is that red and green aren't that great. Part of this may stem from the fact that I've got a bunch of gold cards, and of those, the best are probably in white, with blue coming in second. I notice that during any of the practice solo drafts on TappedOut, I more often than any other, draft blue white. If anyone else has some idea of how to make the other three colors better, I'd LOVE to hear it. I think a good bit of the problem is that blue and white have the most depth, while green lends itself to little more than ramp and red to little beside burn and black does discard and spot removal well at common, but little else.\n\nIn addition, I think I may need to cut a good section of the gold cards out, though I'm not sure how best to go about that. If you notice anything you think would likely underperform, let me know.\n\ntl;dr: Need help on a pauper cube. Gimme ideas.","And I need some help. At the moment, I'm worried some of the strategies I've enabled in it aren't as viable as I may think they are or as well supported as I think. In addition, I feel like I may be missing out on some amazing cards that could make for really interesting play. Becaus

In [28]:
model_checkpoint = "distilgpt2"

In [29]:
print(type(datasets))
print(type(datasets['train']))
datasets.keys()

<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.arrow_dataset.Dataset'>


dict_keys(['train'])

In [30]:
datasets = datasets.remove_columns(['author','body','normalizedBody','id','subreddit','subreddit_id','summary'])
print(type(datasets))

<class 'datasets.dataset_dict.DatasetDict'>


In [31]:
show_random_elements(datasets['train'])

,content
0,"Maybe it's because people don't need to buy a Wii U to get a ton of enjoyment out of it. In my eyes, only 1 or 2 friends in your immediate (local) group really needs it. If it had a great online base, it would lead to unique opportunities that could only be experienced if you bought the system (progression-based multiplayer/desired ranks), but you don't need to buy a wii u to have fun with mario kart, mario bros, or smash bros, you just need that one friend to have one for you."
1,"yes, definitely, I have searched for years to find the best way to play my digital movies. (TOR). In the past i used to use 1tb portable hard drives and usb straight into the tv and then browse the files and play the movies from there... but too many movies had different formats (avi,mkv,etc). Oh man and it sucked getting 80% through a movie and then the movie stopped with a ""unsupported file cannot play video"" which would make my wife and I finish the movie on the computer. Then I found PLEX.. loved it, bought a lifetime sub, never looking back. I have 3 Roku 3's and my brother has 4 chromecasts and they all work well. Plex enables your computer to stream the movies to all your devices simultaneously. if you have any questions with the process let me know."


In [32]:
datasets['train'][0]

{'content': "I think it should be fixed on either UTC standard or UTC+1 year around, with the current zone offsets. \n Moving timescales add a lot of complexity to the implementation of timekeeping systems and have [dubious value]( \n I think seasonal shifting time made sense in the pre-electric past, when timekeeping was more flexible and artificial light was inefficient and often dangerous. \n Now we have machines that work easily with simple timekeeping rules, and it's more beneficial to spend a small amount on energy for lighting, and save the larger cost of engineering things to work with the complex timekeeping rules, as well as saving the irritation to humans. \n Lighting has gotten much more efficient over time; we can squeeze out a lot more photons per unit of energy from a 2012 CFL or LED than a candle could in 1780, or a lightbulb could in 1950. \n There's a lot of room for improvement in how we use lights as well; as lighting control gets more intelligent, there will be a l

In [33]:
#len(datasets['train'][0]['content'])
#type(datasets['train'][]['content'])

In [34]:
#for letter in datasets['train'][0]['content']:
#  print(letter)
datasets['train'][0]['content']

"I think it should be fixed on either UTC standard or UTC+1 year around, with the current zone offsets. \n Moving timescales add a lot of complexity to the implementation of timekeeping systems and have [dubious value]( \n I think seasonal shifting time made sense in the pre-electric past, when timekeeping was more flexible and artificial light was inefficient and often dangerous. \n Now we have machines that work easily with simple timekeeping rules, and it's more beneficial to spend a small amount on energy for lighting, and save the larger cost of engineering things to work with the complex timekeeping rules, as well as saving the irritation to humans. \n Lighting has gotten much more efficient over time; we can squeeze out a lot more photons per unit of energy from a 2012 CFL or LED than a candle could in 1780, or a lightbulb could in 1950. \n There's a lot of room for improvement in how we use lights as well; as lighting control gets more intelligent, there will be a lot of saving

In [35]:
'''
def chunk_examples(example):
  print(example['content'])
  chunks = []

  if len(example) > 500:
    for i,letter in enumerate(example):
      if i > 300 letter == '.' and example[i+1] == ' ':

      chunks += example["content"][0:i+2]
        
  return {"chunks": chunks}

chunked_dataset = datasets.map(chunk_examples, batched=True, remove_columns=None)
'''

'\ndef chunk_examples(example):\n  print(example[\'content\'])\n  chunks = []\n\n  if len(example) > 500:\n    for i,letter in enumerate(example):\n      if i > 300 letter == \'.\' and example[i+1] == \' \':\n\n      chunks += example["content"][0:i+2]\n        \n  return {"chunks": chunks}\n\nchunked_dataset = datasets.map(chunk_examples, batched=True, remove_columns=None)\n'

In [36]:
'''
def chunk_examples(examples):
    chunks = []
    for sentence in examples["content"]:
        chunks += [sentence[i:i + 50] for i in range(0, len(sentence), 50)]
    return {"chunks": chunks}

chunked_dataset = datasets.map(chunk_examples, batched=True, remove_columns=None)
'''

'\ndef chunk_examples(examples):\n    chunks = []\n    for sentence in examples["content"]:\n        chunks += [sentence[i:i + 50] for i in range(0, len(sentence), 50)]\n    return {"chunks": chunks}\n\nchunked_dataset = datasets.map(chunk_examples, batched=True, remove_columns=None)\n'

In [37]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, model_max_length=1024)

In [38]:
def tokenize_function(examples):
    return tokenizer(examples["content"])

In [39]:
#type(datasets)
#show_random_elements(datasets['train'])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["content"])

In [41]:
tokenized_datasets["train"][0]

{'input_ids': [40,
  892,
  340,
  815,
  307,
  5969,
  319,
  2035,
  18119,
  3210,
  393,
  18119,
  10,
  16,
  614,
  1088,
  11,
  351,
  262,
  1459,
  6516,
  49005,
  13,
  220,
  198,
  26768,
  1661,
  66,
  2040,
  751,
  257,
  1256,
  286,
  13357,
  284,
  262,
  7822,
  286,
  640,
  19934,
  3341,
  290,
  423,
  685,
  67,
  549,
  699,
  1988,
  16151,
  220,
  198,
  314,
  892,
  21819,
  15852,
  640,
  925,
  2565,
  287,
  262,
  662,
  12,
  31067,
  1613,
  11,
  618,
  640,
  19934,
  373,
  517,
  12846,
  290,
  11666,
  1657,
  373,
  30904,
  290,
  1690,
  4923,
  13,
  220,
  198,
  2735,
  356,
  423,
  8217,
  326,
  670,
  3538,
  351,
  2829,
  640,
  19934,
  3173,
  11,
  290,
  340,
  338,
  517,
  13205,
  284,
  4341,
  257,
  1402,
  2033,
  319,
  2568,
  329,
  12019,
  11,
  290,
  3613,
  262,
  4025,
  1575,
  286,
  8705,
  1243,
  284,
  670,
  351,
  262,
  3716,
  640,
  19934,
  3173,
  11,
  355,
  880,
  355,
  8914,
  262,
  3516

In [42]:
block_size = tokenizer.model_max_length
#block_size = 128

In [43]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [44]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/3848330 [00:00<?, ? examples/s]

In [52]:
lm_datasets["train"][0]

{'input_ids': [40,
  892,
  340,
  815,
  307,
  5969,
  319,
  2035,
  18119,
  3210,
  393,
  18119,
  10,
  16,
  614,
  1088,
  11,
  351,
  262,
  1459,
  6516,
  49005,
  13,
  220,
  198,
  26768,
  1661,
  66,
  2040,
  751,
  257,
  1256,
  286,
  13357,
  284,
  262,
  7822,
  286,
  640,
  19934,
  3341,
  290,
  423,
  685,
  67,
  549,
  699,
  1988,
  16151,
  220,
  198,
  314,
  892,
  21819,
  15852,
  640,
  925,
  2565,
  287,
  262,
  662,
  12,
  31067,
  1613,
  11,
  618,
  640,
  19934,
  373,
  517,
  12846,
  290,
  11666,
  1657,
  373,
  30904,
  290,
  1690,
  4923,
  13,
  220,
  198,
  2735,
  356,
  423,
  8217,
  326,
  670,
  3538,
  351,
  2829,
  640,
  19934,
  3173,
  11,
  290,
  340,
  338,
  517,
  13205,
  284,
  4341,
  257,
  1402,
  2033,
  319,
  2568,
  329,
  12019,
  11,
  290,
  3613,
  262,
  4025,
  1575,
  286,
  8705,
  1243,
  284,
  670,
  351,
  262,
  3716,
  640,
  19934,
  3173,
  11,
  355,
  880,
  355,
  8914,
  262,
  3516

In [45]:
tokenizer.decode(lm_datasets["train"][0]["input_ids"])

'I think it should be fixed on either UTC standard or UTC+1 year around, with the current zone offsets. \n Moving timescales add a lot of complexity to the implementation of timekeeping systems and have [dubious value]( \n I think seasonal shifting time made sense in the pre-electric past, when timekeeping was more flexible and artificial light was inefficient and often dangerous. \n Now we have machines that work easily with simple timekeeping rules, and it\'s more beneficial to spend a small amount on energy for lighting, and save the larger cost of engineering things to work with the complex timekeeping rules, as well as saving the irritation to humans. \n Lighting has gotten much more efficient over time; we can squeeze out a lot more photons per unit of energy from a 2012 CFL or LED than a candle could in 1780, or a lightbulb could in 1950. \n There\'s a lot of room for improvement in how we use lights as well; as lighting control gets more intelligent, there will be a lot of savi

In [46]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [47]:
from transformers import Trainer, TrainingArguments


In [48]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "distilgpt2_finetune_reddit",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)


In [50]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    #eval_dataset=lm_datasets[""],
)

Cloning https://huggingface.co/dwattles/distilgpt2_finetune_reddit into local empty directory.


In [51]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
#model.predict

In [ ]:
'''
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
'''

In [ ]:
trainer.push_to_hub()